In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install gradio

In [217]:
import gradio as gr

In [218]:
import pickle
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier

In [219]:
import warnings

# Filter out a specific warning
warnings.filterwarnings("ignore")

In [220]:
with open("../content/drive/MyDrive/Projects/LoanApproval-Prediction/Saved_pickle_files/model.pkl", 'rb') as f:
    model = pickle.load(f)

In [221]:
with open("../content/drive/MyDrive/Projects/LoanApproval-Prediction/Saved_pickle_files/min_max_dict.json", 'rb') as f:
    min_max_dict = pickle.load(f)

In [222]:
# elements = ['no_of_dependents', 'education', 'self_employed', 'income_annum', 'loan_amount', 'loan_term', 'cibil_score', 'residential_assets_value',
#                        'commercial_assets_value', 'luxury_assets_value', 'bank_asset_value']

In [223]:
cat_dict = {}

cat_dict["education"] = {"Graduate" : 0, "Not Graduate" : 1}
cat_dict["self_employed"] = {"No":0, "Yes":1}

In [224]:
def encode_values(value):
  if value == "Graduate" or value == "Not Graduate":
    return cat_dict['education'][value]
  else:
    return cat_dict['self_employed'][value]

In [225]:
def scale_dict(min_max_dict, element, number):
    x_max = min_max_dict[element]['max']
    x_min = min_max_dict[element]['min']
    result = (number - x_min)/ (x_max - x_min)
    return result

In [226]:
def deployment_pipeline(no_of_dependents, education, self_employed, income_annum, loan_amount, loan_term, cibil_score, residential_assets_value,
                       commercial_assets_value, luxury_assets_value, bank_asset_value):
  elements = ['no_of_dependents', 'education', 'self_employed', 'income_annum', 'loan_amount', 'loan_term', 'cibil_score', 'residential_assets_value',
                       'commercial_assets_value', 'luxury_assets_value', 'bank_asset_value']
  columns = [no_of_dependents, education, self_employed, income_annum, loan_amount, loan_term, cibil_score,
             residential_assets_value, commercial_assets_value, luxury_assets_value, bank_asset_value]

  cat_columns = [education, self_employed]
  for i in cat_columns:
    val = encode_values(i)
    target_index = columns.index(i)
    columns[target_index] = val



  result_list = []
  for i in range(len(columns)):
    result = scale_dict(min_max_dict, elements[i], columns[i])
    result_list.append(result)

  result_array = np.array(result_list).reshape(1, -1)
  y = model.predict(result_array)[0]
  return y

In [227]:
interface = gr.Interface(
    fn=deployment_pipeline,
    inputs=[
        gr.Slider(0, 10),
        gr.Radio(["Graduate", "Not Graduate"]),  # Assuming education is a categorical variable
        gr.Radio(["Yes", "No"]),  # Assuming self_employed is a binary variable
        "number", "number",
        gr.Slider(0, 20),
        gr.Slider(0, 1000),
        "number", "number", "number", "number"
    ],
    outputs="text",
    examples=[[2, 'Graduate', "Yes", 9600000, 29900000, 12, 778, 2400000, 17600000, 22700000, 8000000],
              [2, "Not Graduate",	"Yes",	4100000,	12200000,	8,	417,	2700000,	2200000,	8800000,	3300000]],
    title="Loan Approval Prediction",
    live=True,
    description="The goal of this project is to develop a machine learning model that predicts whether a loan application should be approved or rejected based on various applicant features. The model aims to assist financial institutions in automating the loan approval process, making it more efficient."
)

interface.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://a0373c66eaa396875c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
